# This code registers GAP HUCs into the Spatial Features Registry.  The code is the first run at this and will likely change as a more complete understanding of SFR needs and requirements are established.  There are many notes within the notebook that suggest improvements to the code and the SB structure that we can implement to help support a more fluid SFR process.

# The code highlights how to post large sql queries to gc2.  It also 

In [1]:
import urllib.request as ur
import geopandas as gpd
import subprocess
import requests
from bis2 import gc2
import datetime
import json

#Delete
hucDownload = 'https://usgs-gap-data.s3.amazonaws.com/Nat_Ranges_Ancillary/HUCs_lower48.gdb.zip'

In [34]:
#Note: This code pulls data down to local disk from direct call to file on amazon s3, 
#it would be ideal to use ScienceBase to access the file of interest and also to do processing in memory (WebLink needs a meaninful title to make that consistently work), 
#also should look into using s3 libraries for python (http://boto3.readthedocs.io/en/latest/reference/services/s3.html)
#Somewhere in this step we should let the data type and SB item "drive" the process.  For example determine appropriate 
#process to get file into geodataframe based on file type.

#Direct HUC Download URL
hucDownload ='https://usgs-gap-data.s3.amazonaws.com/Nat_Ranges_Ancillary/HUCs_lower48.gdb.zip'
#Download GAP HUC12 file to local directory
ur.urlretrieve(hucDownLoad, 'HUCs_lower48.gdb.zip')
#In working directory unzips file
subprocess.call(r'"C:\Program Files\7-Zip\7z.exe" x ' + 'HUCs_lower48.gdb.zip' )

('HUCs_lower48.gdb.zip', <http.client.HTTPMessage at 0x1f70065b550>)

In [2]:
#Create GeoDataFrame of GAP HUC12s, from unzipped file
ghuc_gdf = gpd.read_file('HUCs_lower48.gdb', layer='Hucs')

In [3]:
ghuc_gdf.head(5)

,HUC12RNG,STATES,Shape_Area,Shape_Length,geometry
0,030902030300,FL,5.016106e+09,467346.993995,"(POLYGON ((1505886.25 332166.5, 1508386.25 330..."
1,121102080900,TX,1.114051e+09,287217.454218,(POLYGON ((-129847.9686999992 339565.593799999...
2,121102080100,TX,5.572368e+08,213775.315108,"(POLYGON ((-238270.4844000004 355676.25, -2382..."
3,121102080600,TX,4.453596e+08,131953.362328,(POLYGON ((-179788.9061999992 355720.093799999...
4,121102080700,TX,3.380938e+08,112037.983271,(POLYGON ((-149064.7186999992 366221.343799999...


In [4]:
#View current coordinate system of file.  This is just a note and can get rid of this step.
#Currently this spatial file does not have an official crs... something to improve in future iterations
ghuc_gdf.crs

{'datum': 'NAD83',
 'lat_0': 23,
 'lat_1': 29.5,
 'lat_2': 45.5,
 'lon_0': -96,
 'no_defs': True,
 'proj': 'aea',
 'units': 'm',
 'x_0': 0,
 'y_0': 0}

In [5]:
#Transform coordinate system to web mercator
ghuc_gdf = ghuc_gdf.to_crs({'init': 'epsg:3857'}) 
#Print new coordinate system to make sure transformation worked
ghuc_gdf.crs

{'init': 'epsg:3857'}

In [93]:
#Verify Coordinate System is in web mercator (crs: 3857), if not transform, This needs to be improved upon, right now throws an error
if ghuc_gdf.crs['init'] == 'epsg:3857':
    print ('Coordinate System = Web Mercator, crs: 3857')
else:
    oldCrs = ghuc_gdf.crs['init']
    ghuc_gdf.to_crs({'init': 'epsg:3857'})
    newCrs = ghuc_gdf.crs['init']
    print ('Transformed Coordinate System to ' + newCrs + ' from ' + oldCrs)

KeyError: 'init'

In [6]:
# Set up the actions/targets for this particular instance
thisRun = {}
thisRun["instance"] = "DataDistillery"
thisRun["db"] = "BCB"
thisRun["baseURL"] = gc2.sqlAPI(thisRun["instance"],thisRun["db"])
thisRun["schema"] = "sfr"
thisRun["commitToDB"] = False

gc2Key = gc2.gc2Keys["datadistillery_bcb"]
SbUrl = 'https://www.sciencebase.gov/catalog/item/56d496eee4b015c306f17a42'

In [28]:
#This step helps troubleshoot the upload of many large complex features, which can not be uploaded in one run due to timing out of server
#Return GAP HUC records already in SFR into a list called lookup.  Lookup will be used to ensure we don't duplicate records

q_recordToSearch = "SELECT registration->'sourceLookup' as lookup FROM sfr.sfr_poly where ftype= 'https://www.sciencebase.gov/vocab/term/5898adfde4b050e6125b807f' " 
recordToSearch = requests.get(thisRun["baseURL"]+"&q="+q_recordToSearch).json()
p = recordToSearch['features']
lookup = []
for f in p:
    string = f['properties']['lookup']
    if string.startswith('"') and string.endswith('"'):
        string = string[1:-1]  #For some reason these values are coming back with double quotes, this removes outer quote
        lookup.append(string)
lenList = len(lookup)
cntExisting = 0 
existingList = []
    
for row in ghuc_gdf.itertuples():
    #This if statement checks to see if the HUC was already processed, if it was we just record that and move on. 
    if row.HUC12RNG in lookup:
        cntExisting += 1
        existingList.append({'existingHuc': r})
    else:
        #Register a spatial feature
        #Uses the ScienceBase item and data brought into a geodataframe with crs 3857 to register an item

        #def registerSFR (SbUrl, gdf):
        poly = str(row.geometry)
        title = ('GAP_HUC_' + row.HUC12RNG)
        spat_cert = 'https://www.sciencebase.gov/vocab/term/5822257ee4b0b3d9add24304'
        ftype = 'https://www.sciencebase.gov/vocab/term/5898adfde4b050e6125b807f'
              
        #Build JSON for Registration Information field in SFR 
        #We should bring in information from the SB item to help build this out, example is dataDevelopmentDate, 
        thisRegistration = {}
        thisRegistration["sourceName"] = "USGS Biogeographic Characterization Branch"  #In Future Pull this from SB
        thisRegistration["sourceInfoUrl"] = SbUrl # In Future Pull this from SB
        thisRegistration["sourceFileUrl"] = hucDownload # In Future Pull this from SB
        thisRegistration["registrationDate"] = datetime.datetime.utcnow().isoformat() 
        thisRegistration["sourceLookup"] = row.HUC12RNG
        jd = json.dumps(thisRegistration)
        
        q = "insert into sfr.sfr_poly(registration, spat_cert, title, ftype, the_geom) VALUES ('" + jd + "' ,'"  + spat_cert + "' ,'" \
+ title + "' ,'" + ftype + "', ST_GeomFromText('" + poly + "', 3857))"
        payload = "q=%s&key=%s"%(q,gc2Key)
        url= gc2.baseURLs["sqlapi_datadistillery_bcb"]
        
        try:
            d=[]
            r = requests.post(url,data=payload)
            status = str(r.status_code)
            if r.status_code == 200:
                d.append({'status': status, 'id': str(row.HUC12RNG)})
            else:
                d.append({'status': status, 'id': str(row.HUC12RNG)})
                print (str(row.HUC12RNG) + ' failed with code: ' + status)
        except requests.exceptions.HTTPError:
                d.append({'status': status, 'id': str(row.HUC12RNG)})
                print (str(row.HUC12RNG) + ' failed with code: ' + status)
    

031102021101 failed with code: 502
111402070501 failed with code: 502
130302010801 failed with code: 503
030701060402 failed with code: 503
130302010705 failed with code: 503
080500030207 failed with code: 503
031800020606 failed with code: 503
031502030104 failed with code: 503
080403020101 failed with code: 503
120602010803 failed with code: 503
150802000101 failed with code: 503
031300060608 failed with code: 503
130700070507 failed with code: 503
031602010901 failed with code: 503
120602010804 failed with code: 503
031300070502 failed with code: 503
080500020701 failed with code: 503
120800060803 failed with code: 503
150503010501 failed with code: 503
111403040502 failed with code: 503
031300031304 failed with code: 503
130500040912 failed with code: 503
130500041003 failed with code: 503
031300031302 failed with code: 503
130301020804 failed with code: 503
120800070102 failed with code: 503
031300031306 failed with code: 503
120800020901 failed with code: 503
031102020204 failed 

080602020803 failed with code: 503
120301050306 failed with code: 503
120100020607 failed with code: 503
031300031403 failed with code: 503
150802000204 failed with code: 503
120602010802 failed with code: 503
120601020801 failed with code: 503
080500030302 failed with code: 503
031300031404 failed with code: 503
031700020401 failed with code: 503
031800020702 failed with code: 503
031300090102 failed with code: 503
120601020706 failed with code: 503
031700010403 failed with code: 503
120800040901 failed with code: 503
150502020501 failed with code: 503
111402080102 failed with code: 503
120100020104 failed with code: 503
150502020605 failed with code: 503
111402090106 failed with code: 503
150502020606 failed with code: 503
120602011301 failed with code: 503
130301020802 failed with code: 503
150502020506 failed with code: 503
111402030505 failed with code: 503
120602010903 failed with code: 503
111402030402 failed with code: 503
130600110801 failed with code: 503
080500011010 failed 

130700070601 failed with code: 503
030702010801 failed with code: 503
130600111202 failed with code: 503
130600110804 failed with code: 503
120301070109 failed with code: 503
031502010806 failed with code: 503
031800020101 failed with code: 503
130600111707 failed with code: 503
080500030103 failed with code: 503
120100020107 failed with code: 503
111402030403 failed with code: 503
130500040705 failed with code: 503
031502030308 failed with code: 503
120800060801 failed with code: 503
130700070207 failed with code: 503
030701060307 failed with code: 503
120800021002 failed with code: 503
111403040505 failed with code: 503
031502030107 failed with code: 503
031800020306 failed with code: 503
080500011007 failed with code: 503
080500011303 failed with code: 503
111402030503 failed with code: 503
120601050301 failed with code: 503
031602010108 failed with code: 503
130302021102 failed with code: 503
120602011203 failed with code: 503
111403040504 failed with code: 503
150801010302 failed 

031300060408 failed with code: 503
031602010405 failed with code: 503
130500040606 failed with code: 503
031800020102 failed with code: 503
120800060604 failed with code: 503
080602020301 failed with code: 503
030701040806 failed with code: 503
130700070206 failed with code: 503
111403070208 failed with code: 503
130600111706 failed with code: 503
150803010102 failed with code: 503
120800020406 failed with code: 503
120601020712 failed with code: 503
120601020602 failed with code: 503
031300060407 failed with code: 503
080402060704 failed with code: 503
120301070203 failed with code: 503
150503010507 failed with code: 502
150502020702 failed with code: 503
120601020310 failed with code: 503
120301050107 failed with code: 503
120301070107 failed with code: 503
031300060602 failed with code: 503
080500011301 failed with code: 503
111403070203 failed with code: 503
120601020805 failed with code: 503
031300070801 failed with code: 503
111403040303 failed with code: 503
111402040301 failed 

KeyboardInterrupt: 

# Error that keeps coming up:
ConnectionError: HTTPSConnectionPool(host='gc2.datadistillery.org', port=443): Max retries exceeded with url: /api/v1/sql/bcb (Caused by NewConnectionError('
 <requests.packages.urllib3.connection.VerifiedHTTPSConnection object at  0x00000234DBEEC588>: 
 Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))


In [ ]:
#The below numbers should be the same when initially processing the file.  This could be built into an automated test later on.
print ('Length of existing features list: ' + lenList)
print ('Existing features that were not processed: ' + cntExisting)

In [ ]:
df = pd.DataFrame(d)
for index, row in df.iterrows():
    if  row['status'] != '200':
        print row['comid'], '           ', row['coordCnt'], '                       ', row['status']

In [97]:
print (url)

https://gc2.datadistillery.org/api/v1/sql/bcb


In [23]:
#Testing for keeping zip gdb in memory.  
#Code Failed when trying to view specific layer.

import requests
import uuid
import geopandas as gpd
from osgeo import gdal
import fiona

#requests = requests.get()
vsiz = '/vsimem/{}.zip'.format(uuid.uuid4().hex)
gdal.FileFromMemBuffer(vsiz,bytes(mysock))
with fiona.Collection(vsiz, vsi='zip', layer='Hucs') as f:
    gdf =gpd.GeoDataFrame.from_features(f,crs=f.crs)

FionaValueError: No dataset found at path '/vsizip//vsimem/f943e5efbec74f2caa8b60b2c54b11d9.zip' using drivers: *